In [1]:
from utils.dataset import *
from utils.paths import *

path = 'DTM Switzerland 10m v2 by Sonny.tif'
radius = 10000
grid_size = 10
gps_coords = 45.99563, 7.65948
array, observer_pixel, observer_height  = get_masked_data( gps_coords, radius)

peaks_df = get_peaks( array, observer_pixel, observer_height, radius, grid_size)

year = 2021
month = 6
day = 1
date = {'year':year,'month':month,'day':day}
date = datetime.date(year = date['year'], month = date['month'], day = date['day'])

mdf, tdf, amdf  = get_data(gps_coords, observer_height, peaks_df, date)

In [2]:
import plotly.express as px

px.line(x = mdf.bearing, y = mdf.peak_angle)

In [2]:
array[0, 5,5]

1140.0

In [3]:
path = 'DTM Switzerland 10m v2 by Sonny.tif'
radius = 10
gps_coords = 46.07923, 7.29342
array, observer_pixel, observer_height  = get_masked_data(path, gps_coords, radius)
observer_height

2030.5

In [4]:
array[ 0, int( array.shape[1]/2) , int( array.shape[2]/2) ]

2028.5

In [5]:
def get_annotations(tdf):
    annotations = []
    ticks = []
    gdf = tdf.loc[tdf.elevation > 0].groupby('time')
    for hour, df in gdf:
        df = df.sort_values('date').reset_index()
        # print(df.head)
        x = float( df['azimuth' ] )
        y = float( df['elevation' ] )
        text = hour
        grad = float( df.grad )
        ticks.append(dict(
            type = 'scatter',
            x=[ x, x + 2 * grad],
            y=[ y, y - 2] ,
            line = dict( color="white", width=1), 
            # fill = 'toself',
            marker = None,
            mode = 'lines',
        ))
        
        # markers.append(make_marker_dict(x,y,hour))
        annotations.append( dict(
            text = str(text) + ':00',
            x = x,
            y = y,
            xanchor = 'center',
            yanchor = 'middle',
            xshift = 15 * grad,
            yshift = -20,
            showarrow = False,
            font = dict(color = 'white'),
            opacity = 1
        )    )
    return ticks, annotations



In [6]:
peak_lines = dict(
                type = 'scatter',
                x = mdf.bearing,
                y = mdf.peak_angle,
                line = dict(color = 'gold'),
                mode = 'none',
                stackgroup = 'sun',
                fillcolor = 'black'
                # fill = 'tozeroy'
                )
diff_lines = dict(
                type = 'scatter',
                x = mdf.bearing,
                y = mdf.el_diff,
                line = dict(color = 'gold'),
                mode = 'none',
                stackgroup = 'sun',
                fillcolor = 'gold'
                # fill = 'tozeroy'
                )
sun_line = dict(
                type = 'scatter',
                x = mdf.loc[ mdf.daylight == 'day' ].bearing,
                y = mdf.loc[ mdf.daylight == 'day' ].elevation,
                line = dict(
                    color = 'white',
                    width = 0.5),
            )

ticks, annotations = get_annotations(tdf)

pio.templates.default = "simple_white"

fig = go.Figure()

fig.add_traces([
    peak_lines,
    diff_lines,
    sun_line,
])
fig.add_traces(
    ticks
)
# max_y = max( [ max( [mdf_.elevation.max(), mdf_.peak_angle.max()] ) for mdf_ in mdf_list ] )

fig.update_layout( 
    annotations = annotations,
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 22.5, 45.0, 67.5, 90.0, 112.5, 135.0, 157.5, 180.0, 202.5, 225.0, 247.5, 270.0, 292.5, 315, 337.5, 360] ,
        ticktext = ['N', 'ENE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NWN', 'N'],
        range = [45, 315],
    ),
    yaxis = dict(
        range = [0, 60],
        anchor = 'free',
        position = 0.5,
        visible = False,
        scaleanchor = 'x',
        scaleratio = 0.6
    ),
    showlegend=False,
    autosize=True,
    height=600,
    # margin={'t': 50},
    # margin=dict(
    #     l=50,
    #     r=50,
    #     b=100,
    #     t=100,
    #     pad=4
    # ),
    paper_bgcolor="white",
    font = dict( family = 'verdana', size = 12 )
)